# Boilerplate

In [1]:
%load_ext autoreload
%autoreload 2
%pprint off

Pretty printing has been turned OFF


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
%pprint
# !jupyter nbextension enable --py widgetsnbextension

Pretty printing has been turned ON


# Preamble

In [4]:
import sys,os
sys.path.append(os.getcwd()+'/../src/')
sys.path

['/Users/tillrostalski/Git/main/examples',
 '/Users/tillrostalski/.vscode/extensions/ms-toolsai.jupyter-2022.3.1000901801/pythonFiles',
 '/Users/tillrostalski/.vscode/extensions/ms-toolsai.jupyter-2022.3.1000901801/pythonFiles/lib/python',
 '/Users/tillrostalski/.pyenv/versions/3.9.6/lib/python39.zip',
 '/Users/tillrostalski/.pyenv/versions/3.9.6/lib/python3.9',
 '/Users/tillrostalski/.pyenv/versions/3.9.6/lib/python3.9/lib-dynload',
 '',
 '/Users/tillrostalski/.local/share/virtualenvs/main-ZKJ93Kvm/lib/python3.9/site-packages',
 '/Users/tillrostalski/Git/main/src',
 '/Users/tillrostalski/.local/lib/python3.9/site-packages',
 '/Users/tillrostalski/Git/main/examples/../src/']

In [5]:
if not os.path.exists("logs"):
    os.makedirs("logs")

In [6]:
import fhirdrill	as fd
import pandas			as pd
import fhirpy			as fp
import numpy			as np
import fhir.resources	as fr
import json
# from tqdm import notebook as tqdm
from tqdm import tqdm
tqdm.pandas()
from fhirdrill.constants import CONFIG
tqdm.pandas()

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [8]:
DATAPATH=CONFIG.get('DATAPATH')
RESPATH=f"{DATAPATH}/fhir/"

# ------------------------------------------------------------------------

# FHIR Server Connection

### CLI Usage

To be implemented

### From Default Environment File

In [9]:
drill = fd.Drill()

### From Custom Environment File

In [10]:
drill= fd.Drill(envFile='../.env.example')

### Manual client setup

In [11]:
client = fp.SyncFHIRClient("http://127.0.0.1:32112/hapi-fhir-jpaserver/fhir")
drill  = fd.Drill(client)

# ------------------------------------------------------------------------

# FHIR Drill Usage 

## Resources

Resources can be retrieved by their id as input or by providing search parameters.

Note that the input always has to be of type list. 

### get Resource by input

In [22]:
%%capture
# get single Resource by ID
drill.getPatients(["1"])
drill.getConditions(["5"])
drill.getObservations(["9"])

# get multiple resources by their ids
drill.getPatients(["1", "181", "525"])
drill.getConditions(["5", "209", "218"])
drill.getObservations(["9", "10", "13"])


If there is no get method available for your desired resource, use `getResources()` and provide the resourceType and id

In [44]:
%%capture
drill.getResources(["102"],resourceType="CarePlan")

### search for Resource

In [23]:
%%capture
# get all patients
drill.getPatients(searchParams={})

In [8]:
%%capture
drill.getPatients(searchParams={"_id" : "1"})
drill.getPatients(searchParams={"family" : "Koepp"})
drill.getPatients(searchParams={"family" : "Koepp", "_id": "1"})

Chain get Operations to retrieve connected resources.

In [34]:
%%capture
# get Condtiions for patient 1
drill.getPatients(["1"]).getConditions()

# get Observations for patients with family name Koepp
drill.getPatients(searchParams={"family" : "Koepp"}).getObservations()

# get any Condition with a code that has a text "Hypertension" associated with it
# (either in the text, or a display)
drill.getConditions(searchParams={"code:text": "Hypertension"}).getPatients()

In [49]:
%%capture
drill.searchResources(resourceType="ImagingStudy", searchParams={"bodysite:text":"Knee"})

SEARCH > : 100%|██████████| 12/12 [00:01<00:00,  7.91it/s]


,data
0,"{'resourceType': 'ImagingStudy', 'id': '103', 'meta': {'versionId': '1', 'lastUpdated': '2020-06-11T15:26:55.251+00:00', 'source': '#JlcA0bg5xBW1iwXq', 'tag': [{'system': 'https://smarthealthit.org/tags', 'code': 'synthea-5-2019'}]}, 'identifier': [{'use': 'official', 'system': 'urn:ietf:rfc:3986', 'value': 'urn:oid:1.2.840.99999999.38780235.1559319198559'}], 'status': 'available', 'subject': {'reference': 'Patient/1'}, 'encounter': {'reference': 'Encounter/97'}, 'started': '2015-05-20T19:37:54-04:00', 'numberOfSeries': 1, 'numberOfInstances': 1, 'series': [{'uid': '1.2.840.99999999.1.82637698.1559319198567', 'number': 1, 'modality': {'system': 'http://dicom.nema.org/resources/ontology/DCM', 'code': 'DX', 'display': 'Digital Radiography'}, 'numberOfInstances': 1, 'bodySite': {'system': 'http://snomed.info/sct', 'code': '72696002', 'display': 'Knee'}, 'started': '2015-05-20T19:37:54-04:00', 'instance': [{'uid': '1.2.840.99999999.1.1.25902494.1559319198567', 'sopClass': {'system': 'urn:ietf:rfc:3986', 'code': '1.2.840.10008.5.1.4.1.1.1.1'}, 'number': 1, 'title': 'Image of knee'}]}]}"
1,"{'resourceType': 'ImagingStudy', 'id': '24440', 'meta': {'versionId': '1', 'lastUpdated': '2020-06-11T15:40:13.178+00:00', 'source': '#CAOC38xe6RXk4hrV', 'tag': [{'system': 'https://smarthealthit.org/tags', 'code': 'synthea-5-2019'}]}, 'identifier': [{'use': 'official', 'system': 'urn:ietf:rfc:3986', 'value': 'urn:oid:1.2.840.99999999.13865976.1559319179294'}], 'status': 'available', 'subject': {'reference': 'Patient/24279'}, 'encounter': {'reference': 'Encounter/24434'}, 'started': '2017-10-25T14:33:39-04:00', 'numberOfSeries': 1, 'numberOfInstances': 1, 'series': [{'uid': '1.2.840.99999999.1.45433192.1559319179294', 'number': 1, 'modality': {'system': 'http://dicom.nema.org/resources/ontology/DCM', 'code': 'DX', 'display': 'Digital Radiography'}, 'numberOfInstances': 1, 'bodySite': {'system': 'http://snomed.info/sct', 'code': '72696002', 'display': 'Knee'}, 'started': '2017-10-25T14:33:39-04:00', 'instance': [{'uid': '1.2.840.99999999.1.1.32361793.1559319179294', 'sopClass': {'system': 'urn:ietf:rfc:3986', 'code': '1.2.840.10008.5.1.4.1.1.1.1'}, 'number': 1, 'title': 'Image of knee'}]}]}"
2,"{'resourceType': 'ImagingStudy', 'id': '26225', 'meta': {'versionId': '1', 'lastUpdated': '2020-06-11T15:40:52.094+00:00', 'source': '#ddEQrYqvd0vlGIJ8', 'tag': [{'system': 'https://smarthealthit.org/tags', 'code': 'synthea-5-2019'}]}, 'identifier': [{'use': 'official', 'system': 'urn:ietf:rfc:3986', 'value': 'urn:oid:1.2.840.99999999.24942788.1559319182956'}], 'status': 'available', 'subject': {'reference': 'Patient/26022'}, 'encounter': {'reference': 'Encounter/26219'}, 'started': '2016-02-14T14:59:36-05:00', 'numberOfSeries': 1, 'numberOfInstances': 1, 'series': [{'uid': '1.2.840.99999999.1.93040107.1559319182973', 'number': 1, 'modality': {'system': 'http://dicom.nema.org/resources/ontology/DCM', 'code': 'DX', 'display': 'Digital Radiography'}, 'numberOfInstances': 1, 'bodySite': {'system': 'http://snomed.info/sct', 'code': '72696002', 'display': 'Knee'}, 'started': '2016-02-14T14:59:36-05:00', 'instance': [{'uid': '1.2.840.99999999.1.1.22220940.1559319182973', 'sopClass': {'system': 'urn:ietf:rfc:3986', 'code': '1.2.840.10008.5.1.4.1.1.1.1'}, 'number': 1, 'title': 'Image of knee'}]}]}"
3,"{'resourceType': 'ImagingStudy', 'id': '35237', 'meta': {'versionId': '1', 'lastUpdated': '2020-06-11T15:46:58.436+00:00', 'source': '#bgG9OviXI7It3xm3', 'tag': [{'system': 'https://smarthealthit.org/tags', 'code': 'synthea-5-2019'}]}, 'identifier': [{'use': 'official', 'system': 'urn:ietf:rfc:3986', 'value': 'urn:oid:1.2.840.99999999.31346650.1559319197273'}], 'status': 'available', 'subject': {'reference': 'Patient/34857'}, 'encounter': {'reference': 'Encounter/35231'}, 'started': '2016-02-23T13:09:48-05:00', 'numberOfSeries': 1, 'numberOfInstances': 1, 'series': [{'uid': '1.2.840.99999999.1.32893846.1559319197273', 'number': 1, 'modality': {'system': 'http

# Reference to Resource

# Serialization and Element Access

In [ ]:
patientResource = patientReference.data.iloc[0].to_resource()

patientResource['name']

patientResource.serialize()['birthDate']

In [ ]:
multiPatientReference.getResources()

In [ ]:
multiPatientResource[:1].pretty

In [ ]:
fd.Drill().getDiagnosticReports(
    ['Patient/996a5ac318f013638a9bc05300053c65059cb81abce384a9eb689cce8e9e88f5']
)[:1].keys

# DataFrame Operations

In [ ]:
multiPatientReference[:1]

multiPatientReference[-1:]

multiPatientReference.values

multiPatientReference.info()

# drill.base.Frame

In [ ]:
fd.base.Frame( [[patientResource]], columns=['data'])

# ------------------------------------------------------------------------

# Extraction

# Patient Extraction

# fhirdrill.extraction.getPatients

In [ ]:
drill.getPatients([
	'4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'
	]
)

In [ ]:
drill.getPatients([
	'4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'
	]
).data.item().serialize()

In [ ]:
for i,e in drill.getPatients(
    ['4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136',
    'f0c1e8c8c98fb84d5c46807313e774b8d4fb3e6d11f98af998890dc06af75aa9']).itertuples():
    
    print (f"Row {i} with Patient ID {e.id}")

In [ ]:
getPatientsAsList=drill.getPatients([
	'4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136',
    'f0c1e8c8c98fb84d5c46807313e774b8d4fb3e6d11f98af998890dc06af75aa9']).cast('list')

getPatientsAsList

In [ ]:
[e.pop().id for e in getPatientsAsList]

# Patients as Operand and Extracting other Resources 

In [ ]:
patients = drill.getPatients(
    ['0204dbe820f8ce886ef30a86875e3e58c7e365c728c5ea7ba32d401d240dca91',
    '03bcdc4c13053ea970ef0bed200fc8be36cf0b816e97563b6c9e1faab4217071',
    '4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136',
    'f0c1e8c8c98fb84d5c46807313e774b8d4fb3e6d11f98af998890dc06af75aa9']
)



# fhirdrill.extraction.getRootPatient

In [ ]:
drill.getRootPatients(
    ['0204dbe820f8ce886ef30a86875e3e58c7e365c728c5ea7ba32d401d240dca91',
    '03bcdc4c13053ea970ef0bed200fc8be36cf0b816e97563b6c9e1faab4217071',
    '4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136',
    'f0c1e8c8c98fb84d5c46807313e774b8d4fb3e6d11f98af998890dc06af75aa9']
)

In [ ]:
patients.getRootPatients()

In [ ]:
drill.getResources(
	['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136']
).getRootPatients()

In [ ]:
rootPatients = patients.getResources().getRootPatients().explode('data')

rootPatients.gatherSimplePaths(['name.given','name.family'])

rootPatients.gatherSimplePaths(['id'])

In [ ]:
rootPatients.data[0].name

In [ ]:
for link in rootPatients.data[0]['link']:
	lpat=link.other.to_resource()
	print(lpat.id)

# fhirdrill.extraction.getLinkedPatients

In [ ]:
drill.getLinkedPatients(
	['03bcdc4c13053ea970ef0bed200fc8be36cf0b816e97563b6c9e1faab4217071']
)

# fhirdrill.extraction.getConditions

# Conditions

In [ ]:
conditionsJ84 = drill.getConditions(
	searchParams={
		'code':'J84',
		'_sort': '-onset-date',
		# 'identifier': 'https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation/PrimaryTumorDiagnosis|',
		# '_include':'Condition:subject'
	}
)

conditionsJ84

# Patients for Conditions

In [ ]:
conditionsJ84.getPatients().getLinkedPatients().explode('data')


In [ ]:
conditionsJ84.getPatients().getLinkedPatients().explode('data').data.apply(lambda x:x.id).to_csv(f"{RESPATH}data.ignore")

In [ ]:
conditionsJ84 = pd.read_csv( f"{RESPATH}data.ignore", index_col=0)
conditionsJ84

!head "{RESPATH}data.ignore" 

conditionsJ84.shape

In [ ]:
conditionsJ84.data.iloc[0]

In [ ]:
conditionsJ84 = pd.read_csv(f"{RESPATH}data.ignore", index_col=0)
conditionsJ84

# Conditions for Patient

In [ ]:
drill.getPatients(
    ['4be009b9a298bea867bf1fa84f49ef25fa4eb84d7adc985e6a0cec8dcd68564b']
).getLinkedPatients().explode('data').getConditions()[:5].explode('data')[:5]

# fhidrill.extraction.getEpisodesOfCare

In [ ]:
rootPatients[:2]

In [ ]:
drill.getPatients(
    ['c52ed1efb1adb4db9195c1fa358e9c90e80496ad9ddbc377d2e7166aedd6fd1c']
).getLinkedPatients().explode('data').getEpisodesOfCare()

# fhidrill.extraction.getFamilyMemberHistories

In [ ]:
drill.getPatients(
    ['Patient/97e3bd1445339f7f5890ce1534b95232939a7f40d4093ea578e00b589026a2c3']
).getFamilyMemberHistories()

# fhidrill.extraction.getMedicationAdministrations

In [ ]:
drill.getPatients(
    ['Patient/faa55df4275923276467c9435cd747693816edbf2176905951ccddf3aaa8d0fb'],
).getMedicationAdministrations()[:5]

# fhidrill.extraction.getEpisodesOfCare

In [ ]:
drill.getPatients(
    ['Patient/faa55df4275923276467c9435cd747693816edbf2176905951ccddf3aaa8d0fb'],
).getMedicationRequests().explode("data")[:5]

# fhidrill.extraction.getObservations

In [ ]:
drill.getConditions(searchParams={"code":"J84"}).getPatients(searchParams={"family":"Raddatz"})

In [ ]:
# drill.getObservations(
#     searchParams={"identifier":"https://uk-essen.de/LAB/Nexus/Swisslab/SarsCoV2|"}
# )

drill.getPatients(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'],
).getObservations(
    searchParams={"identifier":"https://uk-essen.de/LAB/Nexus/Swisslab/SarsCoV2|"}
)

# fhirdrill.extraction.getDiagnosticReports

In [ ]:
paths=[
	"subject.reference",
	"presentedForm.contentType",
	"presentedForm.url",
	"presentedForm.title",
	"presentedForm.creation"
]

diagnosticReports=drill\
	.getPatients(["Patient/996a5ac318f013638a9bc05300053c65059cb81abce384a9eb689cce8e9e88f5"])\
		.getDiagnosticReports()

In [ ]:
diagnosticReports.explode("data")[20:25].gatherSimplePaths(paths)

# fhirdrill.extraction.getURLBytes

In [ ]:
paths=[
    'id',
    'subject.reference',
    'presentedForm.url'
]

diagnosticReports = drill.getPatients(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136']
).getDiagnosticReports()[:10]

diagnosticReports=diagnosticReports.gatherSimplePaths(paths)
diagnosticReports=diagnosticReports.explode('presentedForm.url')

diagnosticReports['path']=diagnosticReports['id']+'_'+diagnosticReports['presentedForm.url'].str.split('/').str[-1:].str[0]
diagnosticReports['data']=diagnosticReports.getURLBytes(operateOnCol='presentedForm.url')
diagnosticReports.sendBytesToFile()

In [ ]:
diagnosticReports.loc[:, diagnosticReports.columns!='data'][:5]

# fhirdrill.extraction.base.getAbsolutePaths

In [ ]:
absolutePaths=[
#    'Condition.code.coding.code',
   'Condition.id',
#    'Condition.subject.reference',
    'Encounter.id',
    'Encounter.participant.individual.reference',
#     'Procedure.code.coding.display',
#     'Procedure.code.coding.code',
    'Procedure.status',
    'Procedure.performedDateTime'
]

result=drill.getPatients(
    ['Patient/996a5ac318f013638a9bc05300053c65059cb81abce384a9eb689cce8e9e88f5']
).getAbsolutePaths(absolutePaths)

In [ ]:
# result['Condition']
result['Encounter'].explode(
	'Encounter.participant.individual.reference'
).rename(
	columns={"Encounter.participant.individual.reference":"data"}
).dropna().getResources()[:5]
# result['Procedure']

# ------------------------------------------------------------------------

# Transformation

# fhirdrill.transformation.base.gatherKeys

In [ ]:
# keyCounts=patsC61TumorsResourcesRefs\
# 	.sample(500)\
# 		.resourcesReferences\
# 			.progress_apply(lambda x:list(crawl(x.to_resource().serialize())))\
# 				.explode()\
# 				.value_counts()\
# 					.rename_axis('key')\
# 						.reset_index(name='count').to_frame()

patients.gatherKeys().data.explode().value_counts()[:10]

In [ ]:
drill.getPatients(
    ['4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136']
).gatherKeys(['valueString'])

# fhirdrill.transformation.base.valuesForKeys

In [ ]:
drill.getPatients(
    ['4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'],
    includeLinkedPatients=True
).explode('data').gatherValuesForKeys(['valueString'])

# fhirdrill.transformation.base.gatherReferences

In [ ]:
drill.gatherReferences(
    ['Patient/eca5a0cfd0ac6914e0d524c2467cac7bb7522ba96b781f9dfeff966d309efdb5']
    , recursive=True
)

# fhirdrill.transformation.base.gatherText

In [ ]:
fd.Drill().getPatients(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'],
).getObservations()[:5].gatherText()

# fhirdrill.transformation.base.gatherDates

In [ ]:
fd.Drill().getPatients(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'],
).getObservations()[:5].gatherValuesForKeys(['issued'])

In [ ]:
fd.Drill().getPatients(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'],
).getObservations()[:5].gatherDates()

# ------------------------------------------------------------------------

# Load

In [ ]:
drill.validate(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136']
).sendResourcesToFiles(['./data.ignore'])

# ------------------------------------------------------------------------

# Utils

# Server Profiling

In [ ]:
drill.countServerResources()

# Validation

In [ ]:
drill.validate(
    ['Patient/4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136']
)


# ------------------------------------------------------------------------

# Customization

# fhirdrill.custom.extraction.ship

In [ ]:
drill.getPatients(
    ['Patient/996a5ac318f013638a9bc05300053c65059cb81abce384a9eb689cce8e9e88f5']
).getDiagnosticReportsSHIP()[:5]

# fhirdrill.custom.extraction.ship.getDiagnosticReportFilesSHIP

In [ ]:
drill.getPatients(
    ['4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136'],
    includeLinkedPatients=True
).getDiagnosticReportsSHIP()[:10].getDiagnosticReportsFilesSHIP()

# drill.custom.extraction.ship.getTumorIdsSHIP

In [ ]:
fd.Drill().getPatients(['Patient/996a5ac318f013638a9bc05300053c65059cb81abce384a9eb689cce8e9e88f5'])\
.getEpisodesOfCare()\
.gatherSimplePaths(['identifier.value'])\
.explode('identifier.value')['identifier.value'].unique()

In [ ]:
fd.Drill().getPatients(
    ["4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136"], 
    includeLinkedPatients=True
).explode("data").getTumorIdsSHIP()

# drill.extraction.list.getList

In [ ]:
fd.Drill().getLists(
	searchParams={
	'identifier':'https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation|23288587.266308',
	}
).data.values[0].serialize()['entry'][:5]

# drill.custom.extraction.ship.getTumorResourcesSHIP

In [ ]:
fd.Drill().getPatients(
    ["4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136"]
).getTumorIdsSHIP().getTumorResourcesSHIP()[:1]


In [ ]:
fd.Drill().getPatients(["4137ebc9d2d745fbcf7ab6ea86d626a6d97096a3384dc53d666c3611c16b8136"])\
.getTumorIdsSHIP()\
.getTumorResourcesSHIP()\
.data\
.apply(lambda x:x.serialize()['entry'][:5])


# ------------------------------------------------------------------------

# FHIR Drill Internals

## FHIRPy

## URL Parsing

In [ ]:
import fhirpy.base.utils as fpu
fpu.parse_pagination_url('/app/FHIR/r4/Condition?code=C61&_count=1&_sort=-onset-date&identifier=https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation/PrimaryTumorDiagnosis%7C&_shipPagination=eyJvZmZzZXQiOjIwfQ%3D%3D')

## Raw fetch_resource()

In [ ]:
drill.client._fetch_resource(url[0],url[1]).entry

## Raw client.execute()

In [ ]:
a=drill.client.execute('/app/FHIR/r4/Condition',
#/app/FHIR/r4/Condition?code=C61&_sort=-onset-date&identifier=https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation/PrimaryTumorDiagnosis|&_shipPagination=eyJvZmZzZXQiOjIwfQ%3D%3D',
method='get',
params={
	# 'code':'C61',
	# '_sort':'-onset-date',
	# 'identifier':'https://uk-essen.de/HIS/Cerner/Medico/TumorDocumentation/PrimaryTumorDiagnosis|',
	# '_shipPagination':'eyJvZmZzZXQiOjIwfQ==',
	'_count': 0
}
)
# .total

In [ ]:


a=os.path.dirname(fd.__file__)

